# Trabajo Práctico final - NLP
## Descripción del chatbot
Nuestro chatbot será un especialista en psicología infantil. Tendrá a su disposición en la base de datos vectorial los siguientes datos:
- Texto: Cuenta con archivos pdf de libros de psicología infantil
- Grafos: Usamos una base de datos de grafos online (DBPedia) para buscar libros de interés del usuario.

NOTA: Los datos tabulares tienen ciertas restricciones en una base de datos vectorial: es posible solo acceder a registros individuales de estos, no podemos solicitar a la base de datos que realize operaciones sobre estos datos.

# Preparación de datos


# Segmentación
Al momento de cargar los textos en la base de datos tendremos un problema: se debe segmentar el texto para que cuando se busque según similaridad se encuentre una sección y no el pdf entero.

Como la temática a tratar (psicología) requiere un contexto grande, la opción más adecuada sería usar segmentación recursiva. De esta manera, podemos controlar la longitud de cada split y además nos aseguramos que no se corte a mitad de una oración.

Se limpió el texto removiendo signos innecesarios y marcas de agua.

In [ ]:
!pip3 install autoawq
!pip install chromadb
!pip install typing-extensions --upgrade

In [ ]:
!pip install PyPDF2 langchain


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
import PyPDF2
import re

file_names = [
    '/content/77 - papel juego desarrollo nino resumen evidencia.pdf',
    '/content/13 - Claves fomentar autocontrol tolerancia frustracion hijos.pdf',
    '/content/11 - Caja Herramientas Educadores Manejo Trauma Infantil.pdf']

output_folder = '/content/'
files_list = []
index_list = []

def split_text_into_parts(text):
    # No need for max_length condition, split the text as is
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, separators=['\n\n','\n'])
    parts = text_splitter.split_text(text)
    return parts

for i, path in enumerate(file_names):
    with open(path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)

        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            if len(text) < 200:
                continue
            # Preprocess the text as needed
            remove_tabs = re.sub(r'\t', ' ', text)
            remove_split_words = re.sub(r'-\n', '', remove_tabs)
            remove_watermark = re.sub('Dirección General de la Familia y el Menor\nConsejería de Políticas Sociales y Familia-Comunidad de Madrid', '', remove_split_words)
            remove_watermark = re.sub(r'Caja de Herramientas Para Educadores Para El Manejo de Trauma Infantil|La Red Nacional para el Estrés Traumático Infantil', '', remove_watermark)
            clean_text = re.sub(r'\.{2,}', '', remove_watermark)

            # Split the text into parts
            text_parts = split_text_into_parts(clean_text)

            # Save each part as a separate text file
            for part_num, part_text in enumerate(text_parts):
                output_file = f'{output_folder}body_{i + 1}_page_{page_num + 1}_part_{part_num + 1}.txt'
                files_list.append(output_file)
                index_list.append(f'{i + 1},{page_num + 1},{part_num + 1}')
                with open(output_file, 'w') as f:
                    f.write(part_text)


# ChromaDB
Para poder realizar RAG, necesitamos, además de contar con los datos y el modelo para vectorizarlos, tener una base de datos vectorial que nos permita hacer las búsquedas. Para esto, utilizamos ChromaDB.
Aquí, vectorizaremos los textos utilizando un modelo de embedding para luego hacer consultas

In [5]:
# import chromadb and create client
import chromadb

client = chromadb.Client()

collection = client.create_collection("psychology")

In [6]:
# Cargar los textos
documents = []
for i, txt in enumerate(files_list):
  with open(txt, 'r') as f:
    data = f.read().rstrip()
  documents.append(data)


## Embeddings

Seleccionar un modelo de embeddings adecuado para nuestro objetivo es una parte fundamental.
Para esto, tenemos que tener en cuenta ciertas cosas:
- Tamaño del modelo: debemos tener en cuenta que el poder de procesamiento del entorno de Colab es limitado, tenemos que tener en cuenta esto en el momento de seleccionar el modelo debido al almacenamiento y a la velocidad de la inferencia. Sin embargo, los modelos de embedding no suelen ser muy pesados.
- Idiomas del modelo: Debido a que es probable que la informacion encontrada sea en ingles, debemos buscar modelos multi idiomas (o language agnostic).
- Performance: El modelo debe tener un buen desempeño.

Luego de buscar, se encontraron una serie de modelos LEALLA los cuales son de interes ya que cubren los requisitos anteriores

https://huggingface.co/setu4993

Sin embargo, la carga del modelo mediante las librerías de Hugging Face es lenta y más aún su inferencia. Es por esto que se decide en su lugar usar universal-sentence-encoder-multilingual de Google, que es más rápido.

In [ ]:
!pip install tensorflow-text tensorflow-hub

In [8]:
import tensorflow_text
import tensorflow_hub as hub

# Cargar Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [9]:
# Calcular embeddings para los documentos
embeddings_list = embed(documents).numpy().tolist() # Convertir a lista para que sea serializable

In [10]:
# import numpy as np

# Save NumPy arrays to a binary file
# np.save('/content/sample_data/embeddings.npy', embeddings_list)

In [11]:
# Agregar los documentos a la base de datos
collection.add(
    documents=documents,
    ids= index_list,
    embeddings=embeddings_list
)

In [12]:
preguntas_propuestas = ["¿Por qué a mi hijo le cuesta sociabilizar en la escuela?",
                        "qué consideraciones tengo que tener en cuenta cuando mi hijo juega con sus compañeros?",
                        "Mi hijo nunca quiere comer la comida que le hago",
                        "Como puedo ayudar a mi hijo si sufrió eventos traumáticos",
                        "Enseñar autocontrol a un niño"]

In [13]:
consulta = "Enseñar autocontrol a un niño"
embedding_consulta = embed([consulta]).numpy().tolist()

In [14]:
def search_database(consulta):
  embedding_consulta = embed([consulta]).numpy().tolist()
  results = collection.query(
  query_embeddings=embedding_consulta, # Aquí pasamos el embedding de la consulta
  n_results=1 # Traemos los 3 resultados más cercanos
  )
  return results['documents'][0]

In [15]:
results = search_database(consulta)
results

['Auto, que proviene del griego, significa por sí solo. Y \ncontrol, más o menos, significa dominio o mando. Es decir, autocontrol sería el mando sobre uno mismo, el dominio que se puede ejercer por sí solo. Fomentar el autocontrol, por tanto, es un objetivo educativo. Para hablar de autocontrol, se tienen que dar varias condiciones:\n1. Percepción de control\nTener percepción de que tenemos el control de la situación. Si una persona está convencida de que el conflicto depende de ella, inmediatamente pondrá los mecanis-mos para percibir el problema, analizarlo, indagar, con-tactar con diferentes fuentes de información. En defini-tiva, afrontar la situación y decidir.2. Toma de decisiones\nTiene que existir más de una alternativa de respuesta. \nNo puedo decidir si respiro o no, no hay alternativa. El autocontrol está muy asociado al proceso de toma de decisiones, donde siempre han de existir varias alterna-tivas de acción.\n3. Consecuencias incompatibles']

# Base de datos de grafos
Como mencionamos anteriormente, utilizaremos una base de datos de grafos online, DBPedia. Accederemos a esta usando el wrapper de Python de SPARQL. Mediante la query de SPARQL, buscamos aquellos objetos que sean libros, recolectamos el título, autor y descripción. Luego, filtramos las descripciones que contengan palabras clave que les damos.

Notamos que usamos un traductor para las palabras clave, ya que la cantidad de libros en idioma español es escasa.

In [16]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install SPARQLWrapper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.9 MB/s eta 0:00:00


In [17]:
!pip install googletrans==3.1.0a0

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.0 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16352 sha256=7e511beb794ca6faa41a778cc7e052b4195aff1b70964a12f4fe04fe050ff461
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0

In [23]:
from SPARQLWrapper import SPARQLWrapper, XML
import xml.etree.ElementTree as ET
from googletrans import Translator

translator = Translator()
def translate_spanish_to_english(word):
    try:
        translation = translator.translate(word, src='es', dest='en')
        return translation.text
    except Exception as e:
        return f"Translation error: {e}"


def search_graph(keyword_list):
  full_list = keyword_list
  english_list = list(map(translate_spanish_to_english, keyword_list))
  full_list += english_list
  # Configure the endpoint of DBpedia and the SPARQL query
  sparql = SPARQLWrapper("https://dbpedia.org/sparql")
  # Creamos los filtros según la lista
  keyword_filters = " || ".join([f'CONTAINS(UCASE(?description), UCASE("{kw}"))' for kw in full_list])

  sparql.setQuery(f"""
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX dbo: <http://dbpedia.org/ontology/>
  PREFIX dbp: <http://dbpedia.org/property/>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  PREFIX dct: <http://purl.org/dc/terms/>

  SELECT ?book ?bookLabel ?author ?authorLabel ?description
  WHERE {{
    ?book rdf:type dbo:Book ;
          dbp:author ?author ;
          #dbo:subject ?subject;
          dct:subject dbc:Psychology_books ;
          rdfs:label ?bookLabel ;
          dbo:abstract ?description .

    ?author rdf:type dbo:Person ;
            rdfs:label ?authorLabel .
    FILTER (
    {keyword_filters})
  }}
  LIMIT 500
  """)
  sparql.setReturnFormat(XML)
  results = sparql.query().convert()

  # Convert the Document object to a string
  xml_string = results.toxml()

  # Parse the XML result
  root = ET.fromstring(xml_string)

  # The namespace to use for extracting data
  namespace = '{http://www.w3.org/2005/sparql-results#}'

  # Check the number of results
  print(len(root.findall(f".//{namespace}result")))
  print('Psychology Books, Authors, and Descriptions\n')
  print('------------------------------------------')

  # Iterate over each result and extract relevant data
  for result in root.findall(f".//{namespace}result"):
      book = result.find(f'.//{namespace}binding[@name="bookLabel"]/{namespace}literal').text
      author = result.find(f'.//{namespace}binding[@name="authorLabel"]/{namespace}literal').text
      description = result.find(f'.//{namespace}binding[@name="description"]/{namespace}literal')
      description = description.text if description is not None else "N/A"

      print(f'("{author}", "has_written", "{book}")')
      print(f'("{book}", "description", "{description}")')
      return f'("{author}","{book}")'

search_graph(['depresion','tristeza','angustia'])

44
Psychology Books, Authors, and Descriptions

------------------------------------------
("Alfred Adler", "has_written", "Prassi e teoria della psicologia individuale")
("Prassi e teoria della psicologia individuale", "description", "Prassi e teoria della psicologia individuale è un libro scritto dallo psicologo Alfred Adler, pubblicato nel 1924.Il libro conserva una notevole importanza, visto che assurge anche al ruolo di trattato-manifesto della dottrina proposta da Adler. Il primo capitolo è scritto dall'autore per orientare il lettore nel percorso introduttivo alle teorie fondamentali e ai risultati ottenuti dalla psicologia individuale. Adler chiarisce innanzitutto il significato della definizione "individuale" e di "individuale comparata", che lascia ben aperte le porte all'approfondimento dei rapporti interpersonali.Nelle pagine seguenti vengono descritte le linee direttive della nuova corrente psicologica, rette dal nuovo concetto di finalismo delle nevrosi, al posto della an

'("Alfred Adler","Prassi e teoria della psicologia individuale")'

## Chatbot

Igual que para el modelo de embedding, se deben tener las mismas consideraciones que antes. Teniendo en cuenta que el entorno de Colab tiene como limite 13 billones de parametros para los modelos, debemos buscar teniendo en cuenta ese limite. Teniendo en cuenta que una opcion multi idioma es mandatoria y debido a un juicio personal entre los modelos que ofrece la comunidad de Hugging Face, se decide que la mejor opcion es la que ofrece OpenBuddy. Openbuddy realiza fine-tuning sobre los principales checkpoints o arquitecturas de LLMs. Logran un buen desempeño haciendo posible la incorporacion de multiples idiomas incluso sobre arquitecturas que no tienen un cuerpo variado de idiomas.

Nota: Para reducir los requisitos y mejorar la velocidad, se elige la version cuantizada del modelo provista por TheBloke. El método de cuantización por preferencia personal es AWQ, al ser uno de los últimos y más óptimos en comparación con los demás.



In [25]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_name_or_path = "TheBloke/openbuddy-zephyr-7B-v14.1-AWQ"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=False)
# Load model
model = AutoAWQForCausalLM.from_quantized(model_name_or_path, fuse_layers=True,
                                          trust_remote_code=False, safetensors=True)


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/549k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.89M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/836 [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/18.1k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

quant_config.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.23G [00:00<?, ?B/s]

Fusing layers...: 100%|██████████| 32/32 [00:09<00:00,  3.41it/s]


# Prompt engineering
En el momento de hacer las consultas al chatbot, nos encontramos con un desafío: el prompt engineering. Esto consiste en darle las instrucciones al chatbot de forma correcta para que tenga el funcionamiento que deseamos. Esto es un desafío debido a que estamos limitados a utilizar modelos pequeños. Además de darle las instrucciones que queremos, se realizan varias cosas más:
- Ejemplos Few-shot: Luego de darle las instrucciones, le brindamos al modelo con algunos ejemplos para que pueda seguir.
- Completar el prompt manualmente: Para que no se desvíe, comenzamos su respuesta con la apertura de llaves para asegurarnos que nos dé un json y nada más.

# Primera etapa
Para esta primera parte, decidimos si el sistema RAG debe consultar a la base de datos vectorial o mediante SPARQL para buscar libros.

In [1]:
prompt = "Mi hijo tiene problemas para sociabilizar"
prompt = "Por qué mi hijo se orina en la cama?"

instructions1 = '''Eres un asistente de psicología. Estás conectado a una base de datos. El usuario hará preguntas.
Debes determinar si el usuario quiere buscar libros o si desea recuperar información de nuestra base de datos.
Si está preguntando explícitamente por libros, devuelve un JSON con un solo campo llamado "libros" que se refiere a una lista de palabras clave basadas en la entrada del usuario.
Si no está preguntando por libros y está haciendo una pregunta sobre psicología, devuelve un JSON con un solo campo "simplificado" que simplifica la pregunta del usuario.
Ejemplo:
User: ¿Por qué mi hijo tiene problemas para hacer amigos en la escuela?
Assistant: {'simplificado': 'hijo tiene problemas para hacer amigos en la escuela'}
User: Mi hijo tiene problemas para dormir.
Assistant: {'simplificado': 'hijo tiene problemas para dormir'}
User: Necesito libros sobre el duelo.
Assistant: {'libros': ['duelo', 'pérdida', 'tristeza']}
User: "Podría recomendarme libros para la ansiedad de mi hijo?"
Assistant: {'libros': ['ansiedad', 'nervios', 'tranquilizar']}
'''
prompt_template=f'''
{instructions}
User: {prompt}
Assistant: {{
'''

print("*** Running model.generate:")

def ask_chatbot(instructions, query, json_mode = False):
  prompt_template=f'''
  {instructions}
  User: {query}
  Assistant:
  '''
  if json_mode:
    prompt_template = prompt_template + '{'
  token_input = tokenizer(
      prompt_template,
      return_tensors='pt'
  ).input_ids.cuda()

  # Generate output
  generation_output = model.generate(
      token_input,
      do_sample=True,
      temperature=0.5,
      top_p=0.95,
      top_k=40,
      max_new_tokens=512
  )

  # Get the tokens from the output, decode them, print them
  token_output = generation_output[0]
  text_output = tokenizer.decode(token_output[token_input.shape[1]:])
  #print("LLM output: {", text_output)
  print(instructions)
  return text_output

output = '{' + ask_chatbot(instructions1, prompt, json_mode = True)
output

NameError: ignored

In [42]:
import json
output = output.replace('\n', '').replace('</s>', '')
diccionario = json.loads(output)
diccionario

{'simplificado': 'hijo se orina en la cama',
 'libros': ['hijo',
  'orinar en la cama',
  'problemas de control',
  'control urinario']}

## Segunda parte
Ahora debemos buscar en la base de datos vectorial o grafos según lo determinado en el paso anterior.

In [43]:

def choose_database(diccionario):
  try:
    query = diccionario['simplificado']
    results = search_database(query)
    return {'type':'database', 'context':results}
  except KeyError:
    try:
      books = diccionario['libros']
      results = search_graph(books)
      return {'type':'books', 'context': results}
    except KeyError:
      return {'type':'Error'}
contexto = choose_database(diccionario)
print(contexto)

{'type': 'database', 'context': ['www.NCTSN.org\n8 | Octubre 2008\nLa Red Nacional de Estrés para el Estrés Traumático Infantil\nwww.NCTSN.org\n9\nCambios que se pudieran observar en niños preescolares: \nAcuérdese que los niños pequeños no siempre pueden comunicar lo que les ha ocurrido o sus emociones. La conducta es \nun mejor indicador y los cambios súbitos de comportamiento pueden ser una señal de haber sido expuestos a trauma.\n• Ansiedad de separarse o apego excesivo a las maestras o cuidadores primarios\n• Regresión a etapas de desarrollo ya dominadas (e.g. habla como bebé o se orina en la cama/accidentes de baño)• Falta de progreso en el desarrollo (e.g. no está progresando al mismo nivel que los compañeros de su edad)• Recrea el evento traumático (e.g. habla,  recrea, o dibuja el evento repetidamente)\n• Dificultad para dormir,  al tomar una siesta o a la hora de dormir (e.g. evita el sueño,  se despierta,  o tiene \npesadillas)']}


## Tercera parte
Como ya tenemos el contexto necesario, ahora volvemos a llamar a nuestro modelo de LLM con el contexto proporcionado.

Si el usuario solicita libros, solo le devolvemos los libros pues darle como contexto los libros al chatbot para que genere la respuesta parece redundante.

In [45]:
instructions2 = '''Eres un asistente de psicología. Para dar una mejor respuesta al usuario se te ha proporcionado con contexto adicional para responder.
Responda al usuario tomando como referencia el contexto, use el contexto para responderle.
Contexto:

'''

def answer_user(context, query):
  if context['type'] == 'books':
    if context['context'] == None:
      return 'Lo siento, no he encontrado libros.'
    else:
      return f'Aquí hay un libro que puede interesarte: \n {context["context"]}'
  if context['type'] == 'database':
    response = ask_chatbot(instructions2 + str(context['context']), query)
    return response

final_answer = answer_user(contexto, prompt)
print(final_answer)

Eres un asistente de psicología. Para dar una mejor respuesta al usuario se te ha proporcionado con contexto adicional para responder.
Responda al usuario tomando como referencia el contexto, use el contexto para responderle.
Contexto:

['www.NCTSN.org\n8 | Octubre 2008\nLa Red Nacional de Estrés para el Estrés Traumático Infantil\nwww.NCTSN.org\n9\nCambios que se pudieran observar en niños preescolares: \nAcuérdese que los niños pequeños no siempre pueden comunicar lo que les ha ocurrido o sus emociones. La conducta es \nun mejor indicador y los cambios súbitos de comportamiento pueden ser una señal de haber sido expuestos a trauma.\n• Ansiedad de separarse o apego excesivo a las maestras o cuidadores primarios\n• Regresión a etapas de desarrollo ya dominadas (e.g. habla como bebé o se orina en la cama/accidentes de baño)• Falta de progreso en el desarrollo (e.g. no está progresando al mismo nivel que los compañeros de su edad)• Recrea el evento traumático (e.g. habla,  recrea, o dibu

Juntando todo, esta sería nuestro chatbot

In [ ]:
user_input = input('User: ')

# Determinamos a qué base de datos consultar
first_output = '{' + ask_chatbot(instructions1, user_input, json_mode = True)
output = first_output.replace('\n', '').replace('</s>', '')
diccionario = json.loads(output)

# Buscamos en la base de datos
second_output = choose_database(diccionario)

# Finalmente, respondemos al usuario
final_answer = ask_chatbot(second_output, user_input)
print(final_answer)


# Conclusión

Mediante la implementación de la técnica RAG, hemos podido complementar el performance de nuestro modelo con fuentes externas al conjunto de entrenamiento de este.
Obviamente se contemplan ciertas limitaciones: El modelo es pequeño como para que siga instrucciones complejas y razone temas de psicología; existe cierta limitación en la vectorización de la base de datos; la base de datos de grafos no tiene mucha información respecto a libros y las descripciones no son muy certeras.